# Scenario 2: A single data scientist participating in an ML competition

MLFlow setup:
- Tracking server: yes, local server
- Backend store: sqlite database
- Artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server

To run this example you need to launch the mlflow server locally by running the following command in the terminal:

mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local

In [30]:
import mlflow

mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [31]:
print(f"get tracking uri: {mlflow.get_tracking_uri()}")

get tracking uri: http://127.0.0.1:5000


In [32]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {'C': 0.1, 'random_state': 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X,y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"get default artifacts URI: {mlflow.get_artifact_uri()}")

2025/09/16 21:38:00 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


get default artifacts URI: ./artifacts_local/1/577dc6a6d2f34986ad463966c3dbbb16/artifacts


Interacting with the Model Registry

In [34]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [35]:
client.list_registered_models()

[]

In [36]:
run_id = mlflow.list_run_infos(experiment_id="1")[0].run_id
run_id

'577dc6a6d2f34986ad463966c3dbbb16'

In [37]:
mlflow.register_model(model_uri=f"runs:/{run_id}/models", name='iris-classifier')

Successfully registered model 'iris-classifier'.
2025/09/16 21:38:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1758083887443, current_stage='None', description='', last_updated_timestamp=1758083887443, name='iris-classifier', run_id='577dc6a6d2f34986ad463966c3dbbb16', run_link='', source='./artifacts_local/1/577dc6a6d2f34986ad463966c3dbbb16/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>